In [ ]:
!pip install -qqq -U transformers datasets accelerate bitsandbytes --progress-bar off
!pip install -qqq -U triton --upgrade --progress-bar off

# Домашняя работа

**Сжатие и ускорению работы LLM с помощью квантизации W8A8-INT**

В данной домашней работе мы выполним квантизацию LLM путем замены линейных слоев на свой линейный слой, в котором веса оригинальной модели квантизованны в `int8`, а во время расчета  `forward` котором выполняется квантизация активаций в `int8`. Для создания своего линейного слоя будем использовать triton kernels от `bitsandbytes`. </br>

План работы:
1. Напишем функцию для квантизации матриц на `pytorch` и сравним ее эффективность с функцией, использующую triton kernels.
2. Напишем функцию на `pytorch`, в котором выполняется перемножение целочисленных матриц, а результат деквантизуется в float16. 
3. На основе анализа скорости напишем свой квантизованный слой с использованием тех функций, которые работают быстрее. 
4. Квантизуем LLM путем замены слоев на квантизованные. Выполним анализ скорости вычислений и генерирующей способности модели после квантизации.

In [ ]:
import os
import time
from typing import Optional, Union, Tuple

import torch
import gc

import datasets
from datasets import load_dataset

from pathlib import Path

import transformers

from bnbtriton.quantize_rowwise import quantize_rowwise
from bnbtriton.int8_matmul_rowwise_dequantize import int8_matmul_rowwise_dequantize

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaTokenizer,
    LlamaTokenizerFast
)
# from huggingface_hub import snapshot_download

# cache директория для хранения файлов, загруженных с hf
# os.environ["HF_HOME"] = "/content/hf_cache"
# os.environ["TRANSFORMERS_CACHE"]= "/content/hf_cache"

def print_memory():
    # Функция измерения затраченной GPU памяти
    device='cuda'
    mem_allocated = torch.cuda.memory_allocated(device=device) / 1024**3
    mem_reserved = torch.cuda.memory_allocated(device=device) / 1024**3
    print(f"allocated: {mem_allocated:,.2f} gb")
    print(f" reserved: {mem_reserved:,.2f} gb")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def time_pytorch_function(func, input):
    # Функция для имерения скорости расчета `func` для входа `input`

    # CUDA IS ASYNC so can't use python time module
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    # Warmup
    for _ in range(5):
        func(*input)

    start.record()
    func(*input)
    end.record()
    torch.cuda.synchronize()
    
    return start.elapsed_time(end)

## bitsandbytes Triton kernels 

Используя методы pytorch, реализуем симметричную квантизацию c построчным параметром масштабирования матрицы `W` в int8, результат сравним с результатом применения функции `quantize_rowwise`.

Параметр масштабирования вычисляется для каждой строки матрицы `W` по формуле: <br>
$\alpha = \max(\vert x \vert)$ 

In [ ]:
W = torch.randn((11008, 4096)).to(dtype=torch.float16, device=torch.device('cuda:0'))

In [ ]:
def torch_quantize_rowwise(W: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    quantfactor = 127.0
    # Здесь ваш код
    return W_int8, W_scale

In [8]:
W_int8_torch, W_scale_torch = torch_quantize_rowwise(W)
W_int8_bnb, W_scale_bnb = quantize_rowwise(W)

assert torch.allclose(W_int8_torch, W_int8_bnb, atol=1.0), 'Quantized matrices do not match'
assert torch.allclose(W_scale_torch, W_scale_bnb), 'Scales do not match'

Подсказка: 


если результаты не совпадают попробуйте перевести значение матрицы и параметров масштабирования в fp32 перед делением.

Измерить скорость выполнения `torch_quantize_rowwise` и `quantize_rowwise`. Какая функция работает быстрее?

In [ ]:
time_pytorch_function(#здесь ваш код)

2.0336639881134033

In [ ]:
time_pytorch_function(#здесь ваш код)

0.5888000130653381

С помощью целочисленного матричного умножения `torch._int_mm` реализовать функцию перемножения двух матриц в int8 с последующей деквантизацией результата умножения в fp16. 

Если необходимо, то формула для вычисления может быть найдена в работе [LLM.int8(): 8-bit Matrix Multiplication for Transformers at Scale](https://arxiv.org/pdf/2208.07339)

In [11]:
X = torch.randn((2048, 4096)).to(dtype=torch.float16, device=torch.device('cuda:0'))
bias = torch.randn(11008).to(dtype=torch.float16, device=torch.device('cuda:0'))
X_int8_torch, X_scale_torch = torch_quantize_rowwise(X)
X_int8_bnb, X_scale_bnb = quantize_rowwise(X)

In [ ]:
def torch_int8_matmul_rowwise_dequantize(
    X_int8_torch, 
    W_int8_torch_transpose, 
    X_scale_torch,
    W_scale_torch,
    bias = None
):
    divfactor = 1.0 / (127.0 * 127.0)
    #здесь ваш код

    if bias is not None:
        #здесь ваш код

    return acc

Для того чтобы сравнить производительность нашей функции с функцией, использующей triton kernels, нам необходимо добавить код для деквантизации в функцию `bnbtriton/src/bnbtriton/int8_matmul_rowwise_dequantize.py` </br>
Если возникнут сложности, то код для добавления может быть найден в репозитории `https://github.com/bitsandbytes-foundation/bitsandbytes/tree/main/bitsandbytes`

In [13]:
out_torch = torch_int8_matmul_rowwise_dequantize(
    X_int8_torch, 
    W_int8_torch.t(), 
    X_scale_torch,
    W_scale_torch,
    bias 
)

out_bnb = int8_matmul_rowwise_dequantize(
    X_int8_torch, 
    W_int8_torch.t(), 
    X_scale_torch,
    W_scale_torch,
    bias
)

assert torch.allclose(out_torch, out_bnb), 'Matmul outputs do not match'

In [14]:
out_torch = torch_int8_matmul_rowwise_dequantize(
    X_int8_bnb, 
    W_int8_bnb.t(), 
    X_scale_torch,
    W_scale_torch,
    bias 
)

out_bnb = int8_matmul_rowwise_dequantize(
    X_int8_bnb, 
    W_int8_bnb.t(), 
    X_scale_torch,
    W_scale_torch,
    bias
)

assert torch.allclose(out_torch, out_bnb), 'Matmul outputs do not match'

Воспользуемся функцией `time_pytorch_function` для того, чтобы измерить скорость матричного произведения посредством `torch_int8_matmul_rowwise_dequantize`, `int8_matmul_rowwise_dequantize`.
Выполним замеры с `bias` и без него.

Какой метод работает быстрее?

In [ ]:
time_pytorch_function(#здесь ваш код)

2.288640022277832

In [ ]:
time_pytorch_function(#здесь ваш код)

0.954367995262146

В дополнение к этому сравним полученные результаты с измерениями для функции `torch.nn.functional.linear` для умножения матриц `X` и `W` в fp16.

В каком формате (fp16 или int8 + деквантизация) быстрее выполняется произведение матриц?

In [ ]:
time_pytorch_function(#здесь ваш код)

1.5360000133514404

In [20]:
gc.collect()
torch.cuda.empty_cache()

## Квантизация LLM

Загружаем LLM. В качестве примера взят `Mistral-7B`. <br>
Если памяти GPU недостаточно можно взять модель меньшего размера, например, `TinyLlama`. <br>

Код проверен для llama подобных моделей. Поэтому если архитектура будет отличаться, то могут потребоваться небольшие корректировки кода, на этапе замены линейных слоев.


In [4]:
# Имя модели на hf
# model_name = "mistralai/Mistral-7B-v0.3"
# model_name = "NousResearch/Llama-2-7b-hf"
# model_name = "meta-llama/Llama-2-7b-hf"

# mistral_models_path = Path('/content').joinpath('Mistral-7B-v0.3')
model_path = Path('/home').joinpath("/home/LLaMA/huggingface/Mistral-7B-v0.3")

In [5]:
# Загрузка модели
# Если для загрузки модели требуется токен hf_token, то предварительно записываем его 
# в файл hf_token.txt

# with open("./hf_token.txt", "r") as f:
#     hf_token = f.read()
# os.environ["HF_TOKEN"] = hf_token

# model_path.mkdir(parents=True, exist_ok=True)
# snapshot_download(
#     repo_id="mistralai/Mistral-7B-v0.3",
#     local_dir=model_path,
#     allow_patterns=[
#         "params.json",
#         "config.json",
#         "model.safetensors.index.json",
#         "model-00001-of-00003.safetensors",
#         "model-00002-of-00003.safetensors",
#         "model-00003-of-00003.safetensors",
#         "tokenizer.json",
#         "tokenizer_config.json",
#         "special_tokens_map.json",
#         "tokenizer.model"]
# )

In [6]:
# Загрузка предобученной модели
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    trust_remote_code = True,
    device_map = 'cuda:0'
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.16s/it]


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if not tokenizer.pad_token_id:
    tokenizer.pad_token = tokenizer.eos_token

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [8]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [9]:
print_memory()

allocated: 14.00 gb
 reserved: 14.00 gb


Проверим генерирующие способности модели, путем генерации ответов на два вопроса.

In [ ]:
questions = [
    "Как дела?",
]

answers = []

for question in questions:
    tokenized_input = tokenizer(
        f"QUESTION: {question}\n ANSWER:",
        return_tensors="pt"
    )
    with torch.no_grad():
        output = model.generate(
            **tokenized_input,
            max_length=50, num_beams=3, early_stopping=True,
        )[0]
    answer = tokenizer.decode(output, skip_special_tokens=True)
    answers.append(answer[:answer.find(".")] + ".")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [11]:
answers

['QUESTION: What is result of 2^5?\n ANSWER: 32\n\nQUESTION: What is result of 2^6?\n ANSWER: 64\n\nQUESTIO.',
 'QUESTION: Как добраться до Сколтеха?\n ANSWER: Сколтех находится в 10 минутах ходьбы от станции метро «Сокольник.']

Измерим уровень перплексии на датасете wikitext2-test. Также измерим время, затраченное на расчет.

In [12]:
import random

# Load and process wikitext2 dataset
def get_wikitext2(nsamples=128, seed=0, seqlen=2048, tokenizer=None):
    # Load test datasets
    testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
    testenc = tokenizer("\n\n".join(testdata['text']), return_tensors='pt')
    trainloader = None
    return trainloader, testenc


# Function to evaluate perplexity (ppl) specifically on the wikitext dataset
def eval_ppl_wikitext(model, testenc, bs=1, device=None):
    # Get input IDs
    testenc = testenc.input_ids

    # Calculate number of samples
    nsamples = testenc.numel() // model.seqlen

    # List to store negative log likelihoods
    nlls = []
    print(f"nsamples {nsamples}")

    # Loop through each batch
    for i in range(0,nsamples,bs):
        if i % 50 == 0:
            print(f"sample {i}")

        # Calculate end index
        j = min(i+bs, nsamples)

        # Prepare inputs and move to device
        inputs = testenc[:,(i * model.seqlen):(j * model.seqlen)].to(device)
        inputs = inputs.reshape(j-i, model.seqlen)

        # Forward pass through the model
        lm_logits = model(inputs).logits

        # Shift logits and labels for next token prediction
        shift_logits = lm_logits[:, :-1, :].contiguous()
        shift_labels = inputs[:, 1:]

        # Compute loss
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.reshape(-1, shift_logits.size(-1)), shift_labels.reshape(-1))

        # Calculate negative log likelihood
        neg_log_likelihood = loss.float() * model.seqlen * (j-i)

        # Append to list of negative log likelihoods
        nlls.append(neg_log_likelihood)

    # Compute perplexity
    ppl = torch.exp(torch.stack(nlls).sum() / (nsamples * model.seqlen))

    # Empty CUDA cache to save memory
    torch.cuda.empty_cache()

    return ppl.item()

# Function to evaluate perplexity (ppl) on a specified model and tokenizer
def eval_ppl(model, tokenizer, device=torch.device("cuda:0")):
    # Set dataset
    dataset = "wikitext2"
    model.seqlen = 2048

    # Print status
    print(f"evaluating on {dataset}")

    # Get the test loader
    _, testloader = get_wikitext2(seqlen=model.seqlen, tokenizer=tokenizer)

    # Evaluate ppl in no grad context to avoid updating the model
    with torch.no_grad():
        ppl_test = eval_ppl_wikitext(model, testloader, 1, device)
    return ppl_test

In [13]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
ppl = eval_ppl(model, tokenizer)
end.record()
torch.cuda.synchronize()
print(ppl)
print(start.elapsed_time(end))

evaluating on wikitext2
nsamples 163
sample 0
sample 50
sample 100
sample 150
5.317403316497803
69887.875


In [ ]:
class BnbLinearW8A8OF16(torch.nn.Module):
    '''
    Линейный слой с квантизованными в int8 весами.
    При расчете forward pass активации квантизуются в int8
    '''

    def __init__(
        self,
        in_features: int,
        out_features: int,
        bias: bool = True,
        scale: Union[torch.tensor, float] = 1.0,
        params_dtype: Optional[torch.dtype] = None,
    ):
        super().__init__()

        # Keep input parameters
        self.in_features = in_features
        self.out_features = out_features
        
        self.register_buffer(
            "weight",
            torch.empty(
                self.out_features,
                self.in_features,
                dtype=torch.int8,
                requires_grad=False,
            ),
        )

        if bias:
            self.register_buffer(
                "bias",
                torch.empty(
                    self.out_features,
                    dtype=torch.float16,
                    requires_grad=False,
                ),                
            )
        else:
            self.register_parameter("bias", None)

        # Одномерный массив параметров масштабирования для каждой строки матрицы весов
        self.register_buffer("weight_scale", torch.ones(out_features))
    
    def forward(self, X_3D):
        X = X_3D.view(-1, X_3D.size(-1))

        # Квантизовать входные активации X, используя функцию `quantize_rowwise`
        # здесь ваш код

        # Вычислить произведение весов на активации с 
        # использованием `int8_matmul_rowwise_dequantize`
        res = #здесь ваш код.view(*X_3D.size()[:-1], -1)
        
        return res

    @classmethod
    def from_linear(
        cls,
        linear: torch.nn.Linear
    ):
        q_linear = cls(
            linear.in_features,
            linear.out_features,
            linear.bias is not None,
        )

        if linear.bias is not None:
            q_linear.bias = linear.bias.clone().half()

        linear_weight = linear.weight.data.clone()
        # Квантизовать веса linear_weight в int8
        #здесь ваш код

        assert (
            linear_weight.min() >= -128 and 
            linear_weight.max() <= 127
        ), "Quantized weight out of range"

        q_linear.weight_scale = weight_scale.contiguous()
        q_linear.weight.data = linear_weight.contiguous()

        return q_linear

    def __repr__(self):
        return f'W8A8Linear({self.in_features}, {self.out_features}, bias={self.bias is not None})'

Допишем процедуру `replace_with_qlinear` для замены линейных слоев в нашей LLM
на квантизованные слои. Блоки `embed_tokens` и `lm_head` оставляем без изменений.

In [ ]:
def replace_with_qlinear(root_module):
    '''
    Процедура для замены линейных слоев в блоках трансформеров модели 
    на квантизованные линейные слои BnbLinearW8A8OF16
    '''

    module_name_dict = {name: module for name, module in root_module.named_modules()}
    for name, module in module_name_dict.items():
        if isinstance(module, torch.nn.Linear):
            ind = name.rfind(".")
            if ind == -1:
                father = module_name_dict[""]
            else:
                father = module_name_dict[name[:ind]]

            #здесь ваш код

            setattr(father, name[ind + 1 :], q_linear)
            print(f"replace layer {name} with {q_linear}")
            del module

In [37]:
replace_with_qlinear(model.model)

replace layer layers.0.self_attn.q_proj with W8A8Linear(4096, 4096, bias=False)
replace layer layers.0.self_attn.k_proj with W8A8Linear(4096, 1024, bias=False)
replace layer layers.0.self_attn.v_proj with W8A8Linear(4096, 1024, bias=False)
replace layer layers.0.self_attn.o_proj with W8A8Linear(4096, 4096, bias=False)
replace layer layers.0.mlp.gate_proj with W8A8Linear(4096, 14336, bias=False)
replace layer layers.0.mlp.up_proj with W8A8Linear(4096, 14336, bias=False)
replace layer layers.0.mlp.down_proj with W8A8Linear(14336, 4096, bias=False)
replace layer layers.1.self_attn.q_proj with W8A8Linear(4096, 4096, bias=False)
replace layer layers.1.self_attn.k_proj with W8A8Linear(4096, 1024, bias=False)
replace layer layers.1.self_attn.v_proj with W8A8Linear(4096, 1024, bias=False)
replace layer layers.1.self_attn.o_proj with W8A8Linear(4096, 4096, bias=False)
replace layer layers.1.mlp.gate_proj with W8A8Linear(4096, 14336, bias=False)
replace layer layers.1.mlp.up_proj with W8A8Linear

In [38]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): W8A8Linear(4096, 4096, bias=False)
          (k_proj): W8A8Linear(4096, 1024, bias=False)
          (v_proj): W8A8Linear(4096, 1024, bias=False)
          (o_proj): W8A8Linear(4096, 4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): W8A8Linear(4096, 14336, bias=False)
          (up_proj): W8A8Linear(4096, 14336, bias=False)
          (down_proj): W8A8Linear(14336, 4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32768, bias=False)
)

In [39]:
gc.collect()
torch.cuda.empty_cache()

С помощью функции `print_memory` измерим изменение потребления памяти GPU после квантизации модели.

Как изменилось потребление памяти?

In [40]:
print_memory()

allocated: 7.79 gb
 reserved: 7.79 gb


Посмотрим ответы модели на те же самые вопросы `questions` после квантизации.

In [ ]:
answers_quant = []

#здесь ваш код

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [43]:
answers_quant

['QUESTION: What is result of 2^5?\n ANSWER: 32\n\nQUESTION: What is result of 2^6?\n ANSWER: 64\n\nQUESTIO.',
 'QUESTION: Как добраться до Сколтеха?\n ANSWER: Сколтех находится в 10 минутах ходьбы от станции метро «Славянски.']

Измерим уровень перплексии и скорость ее расчета для квантизованной модели.

In [ ]:
#здесь ваш код

evaluating on wikitext2
nsamples 163
sample 0
sample 50
sample 100
sample 150


59815.22265625

Как изменилась перплексия и скорость расчета бенчмарка после квантизации?

Проверьте изменится ли скорость расчета бенчмарка если квантизовать только линейные слои в `mlp` блоках трансформера или в `self_attn` блоках трансформера.